## Kubflow 설치 및 설정

아래 노트북은 원본 노트북에 설명을 추가 하였습니다.
- 실제 실행시에 여기의 셀의 "결과 값" 과 일치하는지를 확인하면서 실행 하십시오.
- 만일 에러등이 발생하면 확인하시고 다시 실행 해야 합니다.
    - https://github.com/data-science-on-aws/workshop/blob/master/12_kubeflow/00_03_Setup_Kubeflow.ipynb

# Install and Setup Kubeflow on EKS

Based on these instructions:  https://www.kubeflow.org/docs/aws/deploy/install-kubeflow/

# _This Will Take a Few Minutes, Please Be Patient._

In [1]:
%%bash

source ~/.bash_profile

#### Check environment variables are set with valid values
echo $S3_BUCKET
echo $AWS_CLUSTER_NAME
echo $INSTANCE_ROLE_NAME
echo $INSTANCE_PROFILE_ARN

#### Download the `kfctl` CLI tool
curl --location https://github.com/kubeflow/kfctl/releases/download/v1.0.2/kfctl_v1.0.2-0-ga476281_linux.tar.gz | tar xz
sudo mv kfctl /usr/local/bin

#### Get the latest Kubeflow configuration file

# With Ingress
export CONFIG_URI="https://raw.githubusercontent.com/kubeflow/manifests/v1.0-branch/kfdef/kfctl_aws.v1.0.2.yaml"
#export CONFIG_URI='https://raw.githubusercontent.com/kubeflow/manifests/v1.0.2/kfdef/kfctl_aws.v1.0.2.yaml'
echo "export CONFIG_URI=${CONFIG_URI}" | tee -a ~/.bash_profile

#### Set Kubeflow environment variables 
export KF_NAME=${AWS_CLUSTER_NAME}
echo "export KF_NAME=${KF_NAME}" | tee -a ~/.bash_profile

#cd ~/SageMaker/kubeflow/notebooks/part-3-kubernetes

export KF_DIR=$PWD/${KF_NAME}
echo "export KF_DIR=${KF_DIR}" | tee -a ~/.bash_profile

#### Customize the configuration files
# We'll edit the configuration with the right names for the cluster and node groups before deploying Kubeflow.

mkdir -p ${KF_DIR}
cd ${KF_DIR}

curl -O ${CONFIG_URI}

export CONFIG_FILE=${KF_DIR}/kfctl_aws.v1.0.2.yaml
echo "export CONFIG_FILE=${CONFIG_FILE}" | tee -a ~/.bash_profile

sed -i.bak -e "/region: us-west-2/ a \      enablePodIamPolicy: true" ${CONFIG_FILE}
sed -i.bak -e "s@us-west-2@$AWS_REGION@" ${CONFIG_FILE}
sed -i.bak -e "s@roles:@#roles:@" ${CONFIG_FILE}
sed -i.bak -e "s@- eksctl-kubeflow-aws-nodegroup-ng-a2-NodeInstanceRole-xxxxxxx@#- eksctl-kubeflow-aws-nodegroup-ng-a2-NodeInstanceRole-xxxxxxx@" ${CONFIG_FILE}
sed -i.bak -e "s@eksctl-kubeflow-aws-nodegroup-ng-a2-NodeInstanceRole-xxxxxxx@$INSTANCE_ROLE_NAME@" ${CONFIG_FILE}
sed -i.bak -e 's/kubeflow-aws/'"$AWS_CLUSTER_NAME"'/' ${CONFIG_FILE}

#### Generate the Kubeflow installation files
cd ${KF_DIR}

rm -rf kustomize
rm -rf .cache

kfctl build -V -f ${CONFIG_FILE}

#### Deploy Kubeflow
cd ${KF_DIR}

kfctl apply -V -f ${CONFIG_FILE}

#### Delete the usage reporting beacon
kubectl delete all -l app=spartakus --namespace=kubeflow

#### Change to `kubeflow` namespace
kubectl config set-context --current --namespace=kubeflow

sagemaker-ap-northeast-2-343441690612
cluster
eksctl-cluster-nodegroup-cpu-node-NodeInstanceRole-4DG6CRI38MUD
arn:aws:iam::343441690612:role/eksctl-cluster-nodegroup-cpu-node-NodeInstanceRole-4DG6CRI38MUD
export CONFIG_URI=https://raw.githubusercontent.com/kubeflow/manifests/v1.0-branch/kfdef/kfctl_aws.v1.0.2.yaml
export KF_NAME=cluster
export KF_DIR=/home/ec2-user/SageMaker/workshop/12_kubeflow/cluster
export CONFIG_FILE=/home/ec2-user/SageMaker/workshop/12_kubeflow/cluster/kfctl_aws.v1.0.2.yaml
customresourcedefinition.apiextensions.k8s.io/adapters.config.istio.io created
customresourcedefinition.apiextensions.k8s.io/apikeys.config.istio.io created
customresourcedefinition.apiextensions.k8s.io/attributemanifests.config.istio.io created
customresourcedefinition.apiextensions.k8s.io/authorizations.config.istio.io created
customresourcedefinition.apiextensions.k8s.io/bypasses.config.istio.io created
customresourcedefinition.apiextensions.k8s.io/certificates.certmanager.k8s.io created
cu

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   649  100   649    0     0   2053      0 --:--:-- --:--:-- --:--:--  2053
100 27.9M  100 27.9M    0     0  4849k      0  0:00:05  0:00:05 --:--:-- 6662k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  8801  100  8801    0     0  22002      0 --:--:-- --:--:-- --:--:-- 22002
time="2020-08-01T08:33:26Z" level=info msg="No name specified in KfDef.Metadata.Name; defaulting to cluster based on location of config file: /home/ec2-user/SageMaker/workshop/12_kubeflow/cluster/kfctl_aws.v1.0.2.yaml." filename="coordinator/coordinator.go:202"
time="2020-08-01T08:33:26Z" level=info msg="Running `eksctl version` ..." filename="utils/awsutil.go:71"
time="2020-08-01T08:33:26Z" level=info msg="eksctl version: 0.25.0" filename="ut

## Wait for resource to become available
Monitor changes by running kubectl get all namespaces command.


In [2]:
!kubectl -n kubeflow get all


NAME                                                               READY   STATUS              RESTARTS   AGE
pod/admission-webhook-bootstrap-stateful-set-0                     1/1     Running             0          64s
pod/admission-webhook-deployment-59bc556b94-262bc                  0/1     ContainerCreating   0          14s
pod/alb-ingress-controller-798fbd8c96-m6ghq                        1/1     Running             0          40s
pod/application-controller-stateful-set-0                          1/1     Running             0          97s
pod/argo-ui-5f845464d7-9z8hq                                       1/1     Running             0          65s
pod/centraldashboard-d5c6d6bf-xpvlw                                1/1     Running             0          64s
pod/jupyter-web-app-deployment-544b7d5684-fbw88                    1/1     Running             0          63s
pod/katib-controller-6b87947df8-4p784                              0/1     ContainerCreating   0          48s
pod/katib-

In [3]:
!kubectl -n istio-system get all


NAME                                          READY   STATUS      RESTARTS   AGE
pod/cluster-local-gateway-78f6cbff8d-g4sp7    1/1     Running     0          115s
pod/grafana-68bcfd88b6-b2dhk                  1/1     Running     0          2m
pod/istio-citadel-7dd6877d4d-vjhx4            1/1     Running     0          2m
pod/istio-cleanup-secrets-1.1.6-m2gzm         0/1     Completed   0          118s
pod/istio-egressgateway-7c888bd9b9-cvx49      1/1     Running     0          2m
pod/istio-galley-5bc58d7c89-7lqth             1/1     Running     0          2m
pod/istio-grafana-post-install-1.1.6-lxjwl    0/1     Completed   0          118s
pod/istio-ingressgateway-866fb99878-mv8t8     1/1     Running     0          2m
pod/istio-pilot-67f9bd57b-djdf9               2/2     Running     0          2m
pod/istio-policy-749ff546dd-m5dtz             2/2     Running     0          2m
pod/istio-security-post-install-1.1.6-6rrwc   0/1     Completed   0          118s
pod/istio-sidecar-injector-cc5d